<a href="https://colab.research.google.com/github/mabonmn/Feature_Visulalization/blob/main/P3_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initialisation

## Packages

In [ ]:
import numpy as np
import h5py
import os
import cv2
import plotly.graph_objects as go
import plotly.figure_factory as ff
import pandas as pd
pd.options.plotting.backend = "plotly"


## Drive Mount/ Variable Initialisation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

save_path="/content/temp"
isExists = os.path.exists(save_path)
if not isExists:
    os.makedirs(save_path)
save_path=save_path+'/'
Changepts_significant_x=[]
Changepts_significant_y=[]
Changepts_non_significant_x=[]
Changepts_non_significant_y=[]

Mounted at /content/drive


#Parameters

In [ ]:
vid_initial=1
vid_final=50
tol_min=0.005
tol_max=.05

#H5 File Load

In [ ]:
#Load Your H5 File here: 
#For this visulalisation the only requirement of the h5 file is that it should contain the features of each vidoe. 
file_load= h5py.File('/content/drive/MyDrive/Feature Extraction - Mabon/feature_analysis-P3/eccv16_dataset_tvsum_google_pool5.h5', 'r')

Here is the sample format of the H5 File that we use.

In [ ]:
'''
#After Running this Please rerun H5 Load to reload the file.
with file_load as hdf:
  ky1=np.array(list(hdf.keys()))
  g1=hdf.get(ky1[1])
  g1_item_1=np.array(list(g1.keys()))
  hdf.close()
  '''

'\n#After Running this Please rerun H5 Load to reload the file.\nwith file_load as hdf:\n  ky1=np.array(list(hdf.keys()))\n  g1=hdf.get(ky1[1])\n  g1_item_1=np.array(list(g1.keys()))\n  hdf.close()\n  '

Structure:

In [ ]:
'''
print(ky1)
print(g1_item_1)
'''

'\nprint(ky1)\nprint(g1_item_1)\n'

#Standard Deviation of Features

In [ ]:
features_std=[]
for v in range (vid_initial,vid_final+1):
  features=file_load['video_'+str(v)]['features'][:]
  features=pd.DataFrame(features)
  temp=[]
  for i in range(len(features.columns)):
    temp.append(features[i].std())
  features_std.append(temp)
features_std=pd.DataFrame(features_std)

#Plotting Features That Change and Are Static throgh the Video

In [ ]:
for v in range (vid_initial,vid_final+1):
  features=file_load['video_'+str(v)]['features'][:]
  features=pd.DataFrame(features)

  features_tol_min=[]
  features_label_tol_min=[]
  features_tol_max=[]
  features_label_tol_max=[]

  c1=0
  c2=0
  for k in range(1024):

    if(abs(features_std.loc[v-1].iat[k])<tol_min):

      features_tol_min.append(np.array(features[k]))
      features_label_tol_min.append("Feature No: "+str(k))
      Changepts_non_significant_x.append(v)
      Changepts_non_significant_y.append(k)
      c1+=1

    elif(abs(features_std.loc[v-1].iat[k])>tol_max):
      
      features_tol_max.append(np.array(features[k]))
      features_label_tol_max.append("Feature No: "+str(k))
      Changepts_significant_x.append(v)
      Changepts_significant_y.append(k)
      c2+=1

  print(" Video No: "+str(v)+ " \n   Number of features that are Important:     "+str(c2))
  print("   Number of features that are Not Important: "+str(c1))
  
  try:
    
    fig = ff.create_distplot(features_tol_min, features_label_tol_min, show_rug=False, show_hist=False)
    fig.update_layout(title_text=('Video '+str(v)+" Features with STD Less than "+str(tol_min)))
    fig.write_html(save_path+"Video : "+str(v)+" Min.html")
  except:

    print("NO MIN PLOT FOR VIDEO :"+str(v))

  try:

    fig1 = ff.create_distplot(features_tol_max, features_label_tol_max, show_rug=False, show_hist=False)
    fig1.update_layout(title_text=('Video '+str(v)+" Features with STD More than "+str(tol_min)))
    fig1.write_html(save_path+"Video : "+str(v)+" Max.html")
  except:
    
    print("NO MAX PLOT FOR VIDEO :"+str(v))

 Video No: 1 
   Number of features that are Important:     20
   Number of features that are Not Important: 13
 Video No: 2 
   Number of features that are Important:     16
   Number of features that are Not Important: 6
 Video No: 3 
   Number of features that are Important:     7
   Number of features that are Not Important: 8
 Video No: 4 
   Number of features that are Important:     12
   Number of features that are Not Important: 6
 Video No: 5 
   Number of features that are Important:     22
   Number of features that are Not Important: 74
 Video No: 6 
   Number of features that are Important:     7
   Number of features that are Not Important: 56
 Video No: 7 
   Number of features that are Important:     18
   Number of features that are Not Important: 11
 Video No: 8 
   Number of features that are Important:     7
   Number of features that are Not Important: 20
 Video No: 9 
   Number of features that are Important:     8
   Number of features that are Not Important: 15

#Main Plot to Represent Stand-Out Features

In [ ]:
fig=go.Figure()
fig.add_trace(
    go.Scatter(x=np.array(Changepts_non_significant_x),y=np.array(Changepts_non_significant_y),mode='markers', name='Feratures that do not Change')
)
fig.add_trace(
    go.Scatter(x=np.array(Changepts_significant_x),y=np.array(Changepts_significant_y),mode='markers',name='Fetures that Change')
)


fig.update_layout(
   title=("Visual Representation of Features that Change and Do Not Change in each Video"),
    xaxis_title="Video Number",
    yaxis_title="Feature Number")
fig.show()

#Download Plots

In [ ]:
from google.colab import files
dir_to_zip = save_path
output_filename = 'file1.zip' 
delete_dir_after_download = "Yes" 
os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )
if delete_dir_after_download == "Yes":
    os.system( "rm -r {}".format( dir_to_zip ) )
files.download( output_filename )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>